In [1]:
import os
import json

import boto3
import pandas as pd

In [2]:
bucket = os.environ.get("REARC_BUCKET", "rearc-quest-varsh")

In [3]:
csv_key = "rearc-data-quest/bls/pr.data.0.Current"
pop_key = "rearc-data-quest/population/us_population_all_years.json"
        
print("Bucket:", bucket)
print("BLS key:", csv_key)
print("POP key:", pop_key)
s3 = boto3.client("s3")

Bucket: rearc-quest-varsh
BLS key: rearc-data-quest/bls/pr.data.0.Current
POP key: rearc-data-quest/population/us_population_all_years.json


In [4]:
# Load BLS CSV
obj_bls = s3.get_object(Bucket=bucket, Key=csv_key)
bls_df = pd.read_csv(
    obj_bls["Body"],
    sep=r"\s+",       # <- THIS is the fix
    dtype=str
)
print(bls_df)

         series_id  year period    value footnote_codes
0      PRS30006011  1995    Q01      2.6            NaN
1      PRS30006011  1995    Q02      2.1            NaN
2      PRS30006011  1995    Q03      0.9            NaN
3      PRS30006011  1995    Q04      0.1            NaN
4      PRS30006011  1995    Q05      1.4            NaN
...            ...   ...    ...      ...            ...
37234  PRS88003203  2024    Q03  116.593            NaN
37235  PRS88003203  2024    Q04  116.682            NaN
37236  PRS88003203  2024    Q05  116.686            NaN
37237  PRS88003203  2025    Q01  118.271            NaN
37238  PRS88003203  2025    Q02  118.482            NaN

[37239 rows x 5 columns]


In [6]:
mask = bls_df.apply(
    lambda col: col.map(lambda x: isinstance(x, str) and (x != x.strip()))
)
bls_df[mask.any(axis=1)]

,series_id,year,period,value,footnote_codes


In [8]:
    bls_df["year"] = pd.to_numeric(bls_df["year"], errors="coerce")
    bls_df["value"] = pd.to_numeric(bls_df["value"], errors="coerce")
    
    print("BLS shape:", bls_df.shape)
    bls_df.head(10)

BLS shape: (37239, 5)


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN
5,PRS30006011,1996,Q01,-0.2,NaN
6,PRS30006011,1996,Q02,-0.3,NaN
7,PRS30006011,1996,Q03,-0.1,NaN
8,PRS30006011,1996,Q04,0.2,NaN
9,PRS30006011,1996,Q05,-0.1,NaN


In [9]:
bls_df.isna().sum()

series_id             0
year                  0
period                0
value                 0
footnote_codes    37052
dtype: int64

In [10]:
# Load population JSON
obj_pop = s3.get_object(Bucket=bucket, Key=pop_key)
pop_raw = json.loads(obj_pop["Body"].read())

pop_df = pd.DataFrame(pop_raw)
pop_df["year"] = pd.to_numeric(pop_df["year"], errors="coerce")
pop_df["population"] = pd.to_numeric(pop_df["population"], errors="coerce")

print("POP shape:", pop_df.shape)
pop_df.sort_values("year").head(100)

POP shape: (10, 2)


,year,population
0,2013,316128839
1,2014,318857056
2,2015,321418821
3,2016,323127515
4,2017,325719178
5,2018,327167439
6,2019,328239523
7,2021,331893745
8,2022,333287562
9,2023,334914896


In [11]:
pop_1318 = pop_df[(pop_df["year"] >= 2013) & (pop_df["year"] <= 2018)].copy()

mean_pop = pop_1318["population"].mean()
std_pop  = pop_1318["population"].std(ddof=0)  # population stdev

print("Mean population 2013–2018:", int(mean_pop))
print("Std dev population 2013–2018:", float(std_pop))

pop_1318.sort_values("year")

Mean population 2013–2018: 322069808
Std dev population 2013–2018: 3796119.936934378


,year,population
0,2013,316128839
1,2014,318857056
2,2015,321418821
3,2016,323127515
4,2017,325719178
5,2018,327167439


In [12]:
q = bls_df[bls_df["period"].str.startswith("Q", na=False)].copy()

agg = (
    q.groupby(["series_id", "year"], as_index=False)["value"]
      .sum()
      .rename(columns={"value": "year_sum"})
)

idx = agg.groupby("series_id")["year_sum"].idxmax()
best_years = agg.loc[idx].sort_values("series_id").reset_index(drop=True)

best_years.head(100)

,series_id,year,year_sum
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,705.895
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.400
...,...,...,...
95,PRS32006023,2012,503.276
96,PRS32006031,2021,18.200
97,PRS32006032,2021,17.600
98,PRS32006033,1995,710.851


In [13]:
target = bls_df[
    (bls_df["series_id"] == "PRS30006032") &
    (bls_df["period"] == "Q01")
][["series_id", "year", "period", "value"]].copy()

target.sort_values("year").head()

,series_id,year,period,value
1064,PRS30006032,1995,Q01,0.0
1069,PRS30006032,1996,Q01,-4.2
1074,PRS30006032,1997,Q01,2.8
1079,PRS30006032,1998,Q01,0.9
1084,PRS30006032,1999,Q01,-4.1


In [14]:
joined = target.merge(
    pop_df[["year", "population"]],
    on="year",
    how="left"     # keep all BLS rows, add population when available
)

joined = joined.rename(columns={"population": "Population"})
joined = joined.sort_values("year").reset_index(drop=True)

joined

,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.2,NaN
2,PRS30006032,1997,Q01,2.8,NaN
3,PRS30006032,1998,Q01,0.9,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.5,NaN
6,PRS30006032,2001,Q01,-6.3,NaN
7,PRS30006032,2002,Q01,-6.6,NaN
8,PRS30006032,2003,Q01,-5.7,NaN
9,PRS30006032,2004,Q01,2.0,NaN
